In [3]:
import my_utils2 as mu
import requests
import random
import pandas as pd
from tqdm import tqdm

In [4]:
df = mu.get_rawdata('GOLD')

get summonerId....


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.52s/it]


get puuId.....


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:16<00:00,  1.39s/it]


get match_id....


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:17<00:00,  1.43s/it]


get matches & timeline....


100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [01:51<00:00,  3.11s/it]


In [5]:
def lane_processing(tower_lane, tower_team, lane, team):
    if (tower_lane[0] == lane[0]) and (tower_team == team):
        return 1
    else:
        return 0
    

In [6]:
def assi_cal(x):
    try:
        return ','.join(list(map(lambda y: str(y) ,x['assistingParticipantIds'])))
    except:
        return ''

In [7]:
df_creates = []
for i in range(len(df)):
    try:
        match_id = df.iloc[i]['match_id']
        game_duration = df.iloc[i].matches['info']['gameDuration']
        game_version = df.iloc[i].matches['info']['gameVersion']
        match =  df.iloc[i].matches['info']['participants']

        for j in range(len(df.iloc[i].matches['info']['participants'])):

            try:
                riotIdGameName = match[j]['riotIdGameName'] 
            except:
                riotIdGameName = ''
            try:
                riotIdTagline = match[j]['riotIdTagline']
            except:
                riotIdTagline = ''
            summoner_level = df.iloc[i].matches['info']['participants'][j]['summonerLevel']
            participants_id = df.iloc[i].matches['info']['participants'][j]['participantId']
            champion_name = df.iloc[i].matches['info']['participants'][j]['championName']
            champ_experience = df.iloc[i].matches['info']['participants'][j]['champExperience']
            team_position = df.iloc[i].matches['info']['participants'][j]['teamPosition']
            team_id = df.iloc[i].matches['info']['participants'][j]['teamId']
            win = df.iloc[i].matches['info']['participants'][j]['win']
            kills = df.iloc[i].matches['info']['participants'][j]['kills']
            deaths = df.iloc[i].matches['info']['participants'][j]['deaths']
            assists = df.iloc[i].matches['info']['participants'][j]['assists']
            total_damage_dealt = df.iloc[i].matches['info']['participants'][j]['totalDamageDealtToChampions']
            total_damage_taken = df.iloc[i].matches['info']['participants'][j]['totalDamageTaken']

            df_list = list(map(lambda x: x['events'], df.iloc[i].timelines['info']['frames']))
            event_list = [element for array in df_list for element in array]
            tower_log = list(filter(lambda x: x['type'] == 'BUILDING_KILL', event_list))

            try:
                ft_tower_lane = tower_log[0]['laneType']
                ft_tower_team = tower_log[0]['teamId']
            except:
                ft_tower_lane = 'n'
                ft_tower_team = 'n'
            ft = lane_processing(ft_tower_lane, ft_tower_team, team_position, team_id)

            tower_tmp = list(map(lambda x:(x['laneType'], x['teamId'], x['timestamp']) ,tower_log))

            try:
                laneTower = list(filter(lambda x: (x[0][0] == team_position[0]) & (x[1] == team_id), tower_tmp))[0]
                lane_flag = 1
                laneTowerTime = laneTower[-1]
            except:
                lane_flag = 0
                laneTowerTime = 0
            
            blue_ban = list(map(lambda x: str(x['championId']) , df.iloc[i].matches['info']['teams'][0]['bans']))
            red_ban = list(map(lambda x: str(x['championId']) , df.iloc[i].matches['info']['teams'][1]['bans']))
            ban_list = '|'.join(list(set(blue_ban + red_ban)))
                
            kill_log = list(map(lambda x: list(filter(lambda z:z['type'] == 'CHAMPION_KILL' ,x['events'])), df.iloc[i].timelines['info']['frames']))    
            kill_log = [element for array in kill_log for element in array]   
            k = '|'.join(list(map(lambda x: str(x['killerId']), kill_log)))
            v = '|'.join(list(map(lambda x: str(x['victimId']), kill_log)))
            a = '|'.join(list(map(lambda x: assi_cal(x), kill_log)))
            
            try:
                g15 = df.iloc[0].timelines['info']['frames'][15]['participantFrames'][(str(j+1))]['totalGold']
            except:
                g15 = 0
        
                
                
                
            df_creates.append([match_id,game_duration,game_version,riotIdGameName,riotIdTagline,summoner_level,participants_id,
                              champion_name,champ_experience,team_position,team_id,win,kills,deaths,assists,total_damage_dealt,total_damage_taken,
                              ft,lane_flag,laneTowerTime,ban_list,k,v,a,g15])
    except:
        continue      

columns =['matchId', 'gameDuration', 'gameVersion', 'riotIdGameName','riotIdTagline', 
           'summonerLevel', 'participantId', 'championName', 'champExperience', 
           'teamPosition', 'teamId', 'win', 'kills', 'deaths', 'assists', 'totalDamageDealtToChampions', 'totalDamageTaken',
         'firstDT','laneTower','LaneTowerTime','bans','killerId','victimId','assistId','G15']

df2 = pd.DataFrame(df_creates, columns = columns)

In [8]:
blue_team = df2[df2.teamId == 100]
red_team = df2[df2.teamId == 200]

In [9]:
tmp_red= red_team[['matchId', 'gameVersion', 'teamPosition', 'championName','G15']]
tmp_blue = blue_team[['matchId', 'gameVersion', 'teamPosition', 'championName','G15']]

In [10]:
tmp_red.columns = 'matchId', 'gameVersion', 'teamPosition', 'enemy_champ', 'enemy_G15'
tmp_blue.columns = 'matchId', 'gameVersion', 'teamPosition', 'enemy_champ', 'enemy_G15'

In [11]:
blue_team_merge = pd.merge(blue_team,tmp_red, on=('teamPosition','matchId','gameVersion'))
red_team_merge = pd.merge(red_team, tmp_blue, on=('teamPosition','matchId','gameVersion'))

In [12]:
team_df = pd.concat([blue_team_merge, red_team_merge])

In [13]:
team_df

,matchId,gameDuration,gameVersion,riotIdGameName,riotIdTagline,summonerLevel,participantId,championName,champExperience,teamPosition,...,firstDT,laneTower,LaneTowerTime,bans,killerId,victimId,assistId,G15,enemy_champ,enemy_G15
0,KR_6868420828,2237,13.24.547.9214,서프라이즈,3111,259,1,Aatrox,16063,TOP,...,1,1,832334,777|67|24|56|35|84|360|555|910|421,6|6|6|7|2|4|4|2|8|3|2|9|4|4|6|9|2|6|3|9|8|2|8|...,1|1|1|3|7|10|9|6|3|6|10|2|7|9|1|5|6|5|10|2|4|7...,"|||8|3|2,5|5|1||||10|2,5|2,5|||1,5|7,9|4,5|6,8...",3657,Volibear,6401
1,KR_6868420828,2237,13.24.547.9214,써니썬,KR1,275,2,Vi,19357,JUNGLE,...,0,0,0,777|67|24|56|35|84|360|555|910|421,6|6|6|7|2|4|4|2|8|3|2|9|4|4|6|9|2|6|3|9|8|2|8|...,1|1|1|3|7|10|9|6|3|6|10|2|7|9|1|5|6|5|10|2|4|7...,"|||8|3|2,5|5|1||||10|2,5|2,5|||1,5|7,9|4,5|6,8...",6474,Brand,5428
2,KR_6868420828,2237,13.24.547.9214,조용하면채금임,2006,33,3,Lux,18249,MIDDLE,...,0,1,1239559,777|67|24|56|35|84|360|555|910|421,6|6|6|7|2|4|4|2|8|3|2|9|4|4|6|9|2|6|3|9|8|2|8|...,1|1|1|3|7|10|9|6|3|6|10|2|7|9|1|5|6|5|10|2|4|7...,"|||8|3|2,5|5|1||||10|2,5|2,5|||1,5|7,9|4,5|6,8...",4716,Leblanc,4928
3,KR_6868420828,2237,13.24.547.9214,필이겁나잘함,KR1,172,4,Jhin,19087,BOTTOM,...,0,1,1142971,777|67|24|56|35|84|360|555|910|421,6|6|6|7|2|4|4|2|8|3|2|9|4|4|6|9|2|6|3|9|8|2|8|...,1|1|1|3|7|10|9|6|3|6|10|2|7|9|1|5|6|5|10|2|4|7...,"|||8|3|2,5|5|1||||10|2,5|2,5|||1,5|7,9|4,5|6,8...",6353,Ezreal,5210
4,KR_6868420828,2237,13.24.547.9214,Sun Artist,KR1,363,5,Milio,14046,UTILITY,...,0,0,0,777|67|24|56|35|84|360|555|910|421,6|6|6|7|2|4|4|2|8|3|2|9|4|4|6|9|2|6|3|9|8|2|8|...,1|1|1|3|7|10|9|6|3|6|10|2|7|9|1|5|6|5|10|2|4|7...,"|||8|3|2,5|5|1||||10|2,5|2,5|||1,5|7,9|4,5|6,8...",3575,Xerath,3532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,KR_6688236715,2310,13.17.529.525,,,100,6,Riven,23370,TOP,...,0,1,1252871,54|518|58|35|7|22|59|360|111|421,6|3|7|6|5|6|5|6|3|7|2|8|8|2|7|6|3|7|7|3|5|8|1|...,1|8|3|1|10|1|10|1|6|3|7|3|5|9|2|1|8|2|3|7|9|1|...,"|2|||2,4|||7|1,2|6|3||9,10|4,5|8,9,10||2|8|||4...",6401,Darius,3657
141,KR_6688236715,2310,13.17.529.525,,,491,7,Ekko,19015,JUNGLE,...,0,0,0,54|518|58|35|7|22|59|360|111|421,6|3|7|6|5|6|5|6|3|7|2|8|8|2|7|6|3|7|7|3|5|8|1|...,1|8|3|1|10|1|10|1|6|3|7|3|5|9|2|1|8|2|3|7|9|1|...,"|2|||2,4|||7|1,2|6|3||9,10|4,5|8,9,10||2|8|||4...",5428,Elise,6474
142,KR_6688236715,2310,13.17.529.525,,,356,8,Irelia,21588,MIDDLE,...,0,1,1424686,54|518|58|35|7|22|59|360|111|421,6|3|7|6|5|6|5|6|3|7|2|8|8|2|7|6|3|7|7|3|5|8|1|...,1|8|3|1|10|1|10|1|6|3|7|3|5|9|2|1|8|2|3|7|9|1|...,"|2|||2,4|||7|1,2|6|3||9,10|4,5|8,9,10||2|8|||4...",4928,Zed,4716
143,KR_6688236715,2310,13.17.529.525,,,72,9,Xayah,16241,BOTTOM,...,0,1,989216,54|518|58|35|7|22|59|360|111|421,6|3|7|6|5|6|5|6|3|7|2|8|8|2|7|6|3|7|7|3|5|8|1|...,1|8|3|1|10|1|10|1|6|3|7|3|5|9|2|1|8|2|3|7|9|1|...,"|2|||2,4|||7|1,2|6|3||9,10|4,5|8,9,10||2|8|||4...",5210,Caitlyn,3575


In [14]:
team_df['lane_win'] = team_df.apply(lambda x: 1 if x.G15 > x.enemy_G15 else 0, axis=1)

In [15]:
team_df['first_blood'] = team_df.apply(lambda x: 1 if str(x.participantId) == x.killerId.split('|')[0] else 0, axis=1)

In [16]:
def kill_point_calc(x):
    ka = x.kills + x.assists
    kill_log = x.killerId.split('|')
    try:
        if x.teamId == 100:
            lst = list(filter(lambda x : int(x) < 6, kill_log))
        else:
            lst = list(filter(lambda x : int(x) > 5, kill_log))
    except:
        return 0
    try:
        return round(ka/len(lst), 3)
    except:
        return 0

In [17]:
team_df['kill_point']= team_df.apply(kill_point_calc, axis = 1)

In [18]:
team_df['win'] = team_df.apply(lambda x : 1 if x.win == True else 0, axis = 1)

In [19]:
team_df.iloc[0]

matchId                                                            KR_6868420828
gameDuration                                                                2237
gameVersion                                                       13.24.547.9214
riotIdGameName                                                             서프라이즈
riotIdTagline                                                               3111
summonerLevel                                                                259
participantId                                                                  1
championName                                                              Aatrox
champExperience                                                            16063
teamPosition                                                                 TOP
teamId                                                                       100
win                                                                            1
kills                       

In [20]:
tmp_df = team_df[['matchId', 'championName', 'enemy_champ', 'teamPosition', 'win']]

In [21]:
cnt = tmp_df[['championName', 'enemy_champ', 'teamPosition', 'win']].groupby(['championName', 'enemy_champ', 'teamPosition']).count()

In [22]:
cnt = cnt.rename(columns =  {'win' : 'cnt'})

In [23]:
cnt

cnt
championName enemy_champ teamPosition     
Aatrox       Akali       TOP             1
             Gwen        TOP             1
             Trundle     TOP             1
             Urgot       TOP             1
             Volibear    TOP             1
...                                    ...
Zac          Lux         UTILITY         1
Zed          Irelia      MIDDLE          1
Ziggs        Samira      BOTTOM          1
Zoe          Veigar      MIDDLE          1
Zyra         Xerath      UTILITY         1

[272 rows x 1 columns]

In [24]:
cnt.sort_values(by='cnt', ascending=False)

cnt
championName enemy_champ teamPosition     
Caitlyn      Ezreal      BOTTOM          3
Ezreal       Caitlyn     BOTTOM          3
Fiora        Jax         TOP             2
Gnar         Trundle     TOP             2
Caitlyn      Lucian      BOTTOM          2
...                                    ...
Khazix       Lillia      JUNGLE          1
             Shyvana     JUNGLE          1
Kindred      Gragas      JUNGLE          1
KogMaw       Renata      UTILITY         1
Zyra         Xerath      UTILITY         1

[272 rows x 1 columns]

In [25]:
tmp_df[['championName', 'teamPosition', 'win']].groupby(['championName', 'teamPosition']).sum()

win
championName teamPosition     
Aatrox       TOP             3
Ahri         MIDDLE          2
Akali        MIDDLE          0
             TOP             0
Alistar      UTILITY         2
...                        ...
Zac          UTILITY         1
Zed          MIDDLE          0
Ziggs        BOTTOM          0
Zoe          MIDDLE          0
Zyra         UTILITY         0

[145 rows x 1 columns]

In [26]:
cnt_win = tmp_df[['championName', 'enemy_champ', 'teamPosition', 'win']].groupby(['championName', 'enemy_champ', 'teamPosition']).sum()

In [27]:
cnt_win

win
championName enemy_champ teamPosition     
Aatrox       Akali       TOP             1
             Gwen        TOP             0
             Trundle     TOP             1
             Urgot       TOP             0
             Volibear    TOP             1
...                                    ...
Zac          Lux         UTILITY         1
Zed          Irelia      MIDDLE          0
Ziggs        Samira      BOTTOM          0
Zoe          Veigar      MIDDLE          0
Zyra         Xerath      UTILITY         0

[272 rows x 1 columns]

In [28]:
cnt

cnt
championName enemy_champ teamPosition     
Aatrox       Akali       TOP             1
             Gwen        TOP             1
             Trundle     TOP             1
             Urgot       TOP             1
             Volibear    TOP             1
...                                    ...
Zac          Lux         UTILITY         1
Zed          Irelia      MIDDLE          1
Ziggs        Samira      BOTTOM          1
Zoe          Veigar      MIDDLE          1
Zyra         Xerath      UTILITY         1

[272 rows x 1 columns]

In [29]:
result = cnt.join(cnt_win)

In [30]:
result

cnt  win
championName enemy_champ teamPosition          
Aatrox       Akali       TOP             1    1
             Gwen        TOP             1    0
             Trundle     TOP             1    1
             Urgot       TOP             1    0
             Volibear    TOP             1    1
...                                    ...  ...
Zac          Lux         UTILITY         1    1
Zed          Irelia      MIDDLE          1    0
Ziggs        Samira      BOTTOM          1    0
Zoe          Veigar      MIDDLE          1    0
Zyra         Xerath      UTILITY         1    0

[272 rows x 2 columns]

In [31]:
result.sort_values(by = 'cnt', ascending=False)

cnt  win
championName enemy_champ teamPosition          
Caitlyn      Ezreal      BOTTOM          3    2
Ezreal       Caitlyn     BOTTOM          3    1
Fiora        Jax         TOP             2    0
Gnar         Trundle     TOP             2    1
Caitlyn      Lucian      BOTTOM          2    1
...                                    ...  ...
Khazix       Lillia      JUNGLE          1    0
             Shyvana     JUNGLE          1    0
Kindred      Gragas      JUNGLE          1    1
KogMaw       Renata      UTILITY         1    0
Zyra         Xerath      UTILITY         1    0

[272 rows x 2 columns]

In [32]:
query_mysql = """
create table lol_datas(gameId varchar(30), gameDuration int, gameVersion varchar(40), summonerName varchar(50),
summonerLevel int, participantId int, championName varchar(50), champExperience int,
teamPosition varchar(50), teamId int, win varchar(50), kills int, deaths int, assists int,
totalDamageDealtToChampions int ,totalDamageTaken int, g_5 int,g_6 int,g_7 int,
g_8 int,g_9 int,g_10 int,g_11 int,g_12 int,g_13 int,g_14 int,
g_15 int,g_16 int,g_17 int,g_18 int,g_19 int,g_20 int,g_21 int,
g_22 int,g_23 int, g_24 int,g_25 int
)
"""

In [33]:
conn = mu.connect_mysql()

In [34]:
mu.sql_execute(conn,query_mysql)

OperationalError: (1050, "Table 'lol_datas' already exists")

In [35]:
conn.commit()

In [36]:
conn.close()

In [37]:
query_mysql2 = 'ALTER TABLE lol_datas add primary key (gameId, participantId)'

In [38]:
conn = mu.connect_mysql()

In [39]:
mu.sql_execute(conn,query_mysql2)

OperationalError: (1068, 'Multiple primary key defined')

In [40]:
conn = mu.connect_mysql()
query = 'select * from lol_datas'
result = mu.sql_execute_dict(conn, query)
conn.close()

In [41]:
result_df = pd.DataFrame(result)

In [42]:
result_df.columns

Index(['gameId', 'gameDuration', 'gameVersion', 'summonerName',
       'summonerLevel', 'participantId', 'championName', 'champExperience',
       'teamPosition', 'teamId', 'win', 'kills', 'deaths', 'assists',
       'totalDamageDealtToChampions', 'totalDamageTaken', 'g_5', 'g_6', 'g_7',
       'g_8', 'g_9', 'g_10', 'g_11', 'g_12', 'g_13', 'g_14', 'g_15', 'g_16',
       'g_17', 'g_18', 'g_19', 'g_20', 'g_21', 'g_22', 'g_23', 'g_24', 'g_25'],
      dtype='object')

In [43]:
result_df = result_df[(result_df.gameDuration > 900)]

In [44]:
sample = result_df[['gameId', 'championName', 'win', 'teamPosition', 'g_15', 'teamId']]

# 챔피언 1 vs 1 라인전 승률: G15의 데이터를 바탕으로 승률 구하기

# 챔피언 1 vs 1 게임 승률: G15가 아니라 WIN데이터 바탕으로 승률구하기

In [45]:
sample

,gameId,championName,win,teamPosition,g_15,teamId
0,KR_6637806460,Renekton,False,TOP,4680,100
1,KR_6637806460,Karthus,False,JUNGLE,4657,100
2,KR_6637806460,Zed,False,MIDDLE,5744,100
3,KR_6637806460,Ezreal,False,BOTTOM,6305,100
4,KR_6637806460,Rakan,False,UTILITY,3637,100
...,...,...,...,...,...,...
15653,KR_6870151290,Lucian,True,,20580,100
15654,KR_6870151290,Ryze,False,,15750,200
15655,KR_6870151290,Shaco,False,,15750,200
15656,KR_6870151290,Kayn,False,,15815,200


In [46]:
blue_team = sample[sample.teamId == 100]
red_team = sample[sample.teamId == 200]

In [47]:
tmp_red= red_team[['gameId', 'championName', 'teamPosition','g_15']]
tmp_blue= blue_team[['gameId', 'championName', 'win', 'teamPosition','g_15']]

In [48]:
tmp_red.columns = 'gameId', 'enemy_champ', 'teamPosition', 'enemy_G15'
tmp_blue.columns = 'gameId', 'championName', 'win', 'teamPosition', 'g_15'

In [50]:
team_merge = pd.merge(tmp_blue,tmp_red, on=('gameId','teamPosition'))

In [62]:
team_merge['win'] = team_merge.apply(lambda x : 1 if x.win == 'True' else 0, axis = 1)

In [51]:
team_merge['lane_win'] = team_merge.apply(lambda x: 1 if x.g_15 > x.enemy_G15 else 0, axis=1)

In [54]:
cntt = team_merge[['championName', 'enemy_champ', 'teamPosition', 'lane_win']].groupby(['championName', 'enemy_champ', 'teamPosition']).count()

In [55]:
cntt = cntt.rename(columns =  {'lane_win' : 'cnt'})

In [57]:
win = team_merge[['championName', 'enemy_champ', 'teamPosition', 'lane_win']].groupby(['championName', 'enemy_champ', 'teamPosition']).sum()

In [59]:
resultt = cntt.join(win)

In [64]:
cntt2 = team_merge[['championName', 'enemy_champ', 'teamPosition', 'win']].groupby(['championName', 'enemy_champ', 'teamPosition']).count()

In [66]:
cntt2 = cntt.rename(columns =  {'win' : 'cnt'})

In [67]:
win2 = team_merge[['championName', 'enemy_champ', 'teamPosition', 'win']].groupby(['championName', 'enemy_champ', 'teamPosition']).sum()

In [75]:
resultt2 = cntt2.join(win2)

In [83]:
resultt2.drop('cnt', axis = 1, inplace = True)

In [93]:
result_df = resultt.join(resultt2)

In [96]:
result_df = result_df.rename(columns =  {'cnt' : 'games'})

In [99]:
result_df = result_df.rename(columns =  {'win' : 'win_cnt'})

In [100]:
result_df.sort_values('games', ascending= False)

games  lane_win  win_cnt
championName enemy_champ teamPosition                          
Lucian       Ezreal      BOTTOM           38        19       19
Ezreal       Kaisa       BOTTOM           36        16       17
Kaisa        Ezreal      BOTTOM           34        16       21
Graves       LeeSin      JUNGLE           29        14       17
LeeSin       Graves      JUNGLE           28        19       16
...                                      ...       ...      ...
Kennen       Ornn        TOP               1         1        1
             Nasus       TOP               1         1        0
             Mordekaiser TOP               1         1        1
             Kayle       TOP               1         0        0
Zyra         Zac         UTILITY           1         1        1

[6092 rows x 3 columns]